In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/boydingalls


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8],\
                         row[12], row[13], row[16]))


NameError: name 'file_path_list' is not defined

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

# from cassandra.cluster import Cluster
# cluster = Cluster()

# # To establish connection and begin executing queries, need a session
# session = cluster.connect()

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)
    

In [ ]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [ ]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# 1) primary key(session_id,artist,song_title)
# 2) primary key(session_id, user_id, item_in_session, artist, song_title)

# 3) select first_name, last_name from library where song_title='All Hands Against His Own'
query="drop table if exists evnt_data_new"  
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS evnt_data_new "
query = query + "(sessionid text, iteminsession text, userid text, artist text, song text \
                 , length text, firstname text, lastname text, PRIMARY KEY (sessionid,iteminsession,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for row in csvreader:
        #print(row[4])
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into evnt_data_new (sessionid, iteminsession, userid, artist, song, length, firstname \
                                           , lastname)"
        query=query+"values (%s, %s, %s, %s, %s, %s, %s, %s)"        

        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#        session.execute(query, (line[8], line[10], line[3], line[0],line[9], line[5], line[1], line[4]))
        try:
            session.execute(query, (row[8], row[3], row[10], row[0], row[9], row[5], row[1], row[4]))
        except Exception as e:
            print(e)  
    

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
 
query = "select * from evnt_data_new where sessionId='338'"
try:
    rows = session.execute(query)

except Exception as e:
    print(e)


for row in rows:
    print (row.sessionid, row.iteminsession, row.userid, row.artist, row.song, row.length, 
           row.firstname, row.lastname)
 